## COVID19- Argentina


In [ ]:
#hide
import os
import numpy as np
import pandas as pd
import glob
import locale
import arcovid19 as arc19
import altair as alt
from altair import datum
from jinja2 import Template

#
from IPython.display import HTML
# Set Language
_=locale.setlocale(locale.LC_TIME,'es_ES.utf8')

In [ ]:
#hide
running_nb=True
write_products=False

In [ ]:
#hide

## readers
def get_prc_df_local(data_src,sheets=['casos','casos_fix','casos_vespertinos']):
    """
    """
    lst_df_=[]
    for sn in sheets:
        df_=pd.read_excel(data_src,sheet_name=sn,parse_dates=[0])
        lst_df_.append(df_)
    df_cat_=pd.concat(lst_df_,axis=0,)
    df_cat_['fecha']=pd.to_datetime(df_cat_['fecha'].dt.strftime('%Y-%m-%d'))
    # Group all with "FIX"
    df_cat_=df_cat_.groupby(['provincia','fecha'])[['confirmados','muertes','recuperados']].sum().reset_index()
    #
    df_cumsum_=df_cat_.groupby(['provincia'])[['confirmados','muertes','recuperados']].cumsum().add_prefix('cumsum_')
    # concat dfs
    df_cat=pd.concat([df_cat_,df_cumsum_],axis=1)

    return df_cat


def get_prc_df_arc19(url=arc19.CASES_URL,cached=True):
    """
    """
    #
    df_tmp=arc19.load_cases(cached=cached)
    df_tmp.reset_index(inplace=True)
    df_tmp.drop(['provincia_status','Pcia_status'],axis=1,inplace=True)
    df_tmp.dropna(inplace=True)
    # arc19 nos da los acumulados diarios
    cmra_daily={'C':'confirmados','R':'recuperados','D':'muertes','A':'activos'}
    cmra_cumsum={'C':'cumsum_confirmados','R':'cumsum_recuperados','D':'cumsum_muertes','A':'cumsum_activos'}
    cmra_cumsum_list=list(cmra_cumsum)
    cmra_daily_list=list(cmra_daily)

    df_cat_list=[]

    for provincia in list(set(df_tmp.cod_provincia)):
        #provincia='ARG'
        df_tmp_prov=df_tmp[df_tmp.cod_provincia==provincia].T
        status_names=df_tmp_prov.loc['cod_status'].to_dict()
        df_tmp_prov.rename(columns=status_names,inplace=True)
        df_tmp_prov.drop(['cod_provincia','cod_status'],inplace=True)
        df_tmp_prov.reset_index(inplace=True)
        df_tmp_prov['provincia']=provincia
        df_tmp_prov.rename(columns={'index':'fecha'},inplace=True)
        df_tmp_prov['fecha']=pd.to_datetime(df_tmp_prov['fecha'].dt.strftime('%Y-%m-%d'))
        d=df_tmp_prov[cmra_cumsum_list].diff()
        d.loc[0]=df_tmp_prov[cmra_cumsum_list].loc[0]
        d.rename(columns=cmra_daily,inplace=True)
        df_prov=pd.concat([df_tmp_prov,d],axis=1).rename(columns=cmra_cumsum)
        df_cat_list.append(df_prov)
    df_=pd.concat(df_cat_list)
    provDict=arc19.CODE_TO_POVINCIA
    provDict.update({'ARG':'Argentina_Nacion','BA': 'Buenos Aires'})
    #
    df_['provincia']=df_['provincia'].map(provDict)
    return df_

#

def get_frame_unstack(df_,oncol,group_by_lst=['provincia','fecha']):
    s=df_.groupby(group_by_lst)[oncol].sum().unstack()
    return s

def get_template(path):
    from urllib.parse import urlparse
    if bool(urlparse(path).netloc):
        from urllib.request import urlopen
        return urlopen(path).read().decode('utf8')
    return open(path).read()

def write_report(hmtl,out_name):
    with open(out_name,'w') as f:
        f.write(html)
    
#
provDict={'Ciudad Autónoma de Buenos Aire':'CABA'}


In [ ]:
#hide_input
# CSV sources
local=True # if false import from arcovid
if local:
    data_src_= '../data/casos.xlsx'
    #
    df_ = get_prc_df_local(data_src_)
    df_['provincia'] = df_['provincia'].apply(
        lambda x: provDict[x] if x in provDict else x)
else:
    data_src_=arc19.CASES_URL
    df_=get_prc_df_arc19(url=data_src_,cached=True)
    arc19.load_cases()
print('Importando desde: {}'.format(data_src_))
df_.head()

In [ ]:
#
source=df_
circ=alt.Chart(source).mark_circle(color='orange').encode(
    x=alt.X('fecha:T',axis=alt.Axis(title='Fecha'),),
    y=alt.Y('provincia',axis=alt.Axis(title='Provincias/Nación')),
    size='confirmados',
    tooltip=['confirmados', 'fecha','provincia','muertes']
).properties(width=600, height=600).interactive()


In [ ]:
circ

In [ ]:
#
pts = alt.selection(type="single", encodings=['x'])
rect=alt.Chart(source).mark_rect().encode(
    alt.Y('provincia:O', title='Provincias/Nacion'),
    alt.X('monthdate(fecha):O', title='Fecha'),
    alt.Color('confirmados:Q', title='Casos Confirmados',),
tooltip=['confirmados', 'fecha','provincia','muertes','recuperados']
).properties(width=600, height=600).interactive()
rect

In [ ]:
from scipy import stats
df_reg=source[source.provincia=='Argentina_Nacion'].copy()
#0 based
dayFit=np.arange(6,15,1)
deltaT=dayFit[1]-dayFit[0]
y=df_reg.iloc[dayFit]['cumsum_confirmados'].values
x=dayFit
#
slope, intercept,_,_,_=stats.linregress(x,np.log(y))
#
fitted=np.exp(intercept)*np.exp(slope)**np.arange(0,df_reg.shape[0],1)
#
df_reg['fitted']=np.nan
df_reg['fitted']=fitted
df_reg['fitted_flag']='extrapolado'
df_reg.loc[dayFit,'fitted_flag']='ajustado'
df_reg['daily_expected']=df_reg['fitted'].diff().fillna(0)

In [ ]:
pts = alt.selection(type="single", encodings=['x'],empty='none')
#
points = alt.Chart(source).mark_circle(color='orange').encode(
    alt.X('fecha:T',title='Fecha'),
    alt.Y('cumsum_confirmados:Q',scale=alt.Scale(type='log'),title='#Confirmados Totales en Argentina (log scale)'),
    size=alt.Size('confirmados',title='#Confirmados(dia)'),
    color=alt.condition(pts, 'confirmados:Q', alt.value('orange'),legend=None),
    tooltip=['confirmados', 'fecha','provincia','muertes','recuperados']
).properties(        width=550,
        height=400).transform_filter((datum.provincia == 'Argentina_Nacion')).add_selection(pts)

tooltip_bars=[alt.Tooltip('cumsum_confirmados:Q', title='#Confirmados Totales'),alt.Tooltip('fecha:T', title='Fecha'),alt.Tooltip('provincia:N', title='Provincia')]

bars = alt.Chart(source).mark_bar().encode(
       alt.X('provincia:N',axis=alt.Axis(minExtent=100),title='Provincia de Residencia'),
        alt.Y('cumsum_confirmados:Q',title='Confirmados x Provincia'),
        color=alt.Color('cumsum_confirmados:Q',title='Confirmados x Provincia',legend=None),
        tooltip=tooltip_bars).properties(  
    width=550,
        height=220
    ).transform_filter(datum.provincia != 'Argentina_Nacion').transform_filter((pts)).interactive()

texts = alt.Chart(source).mark_text(dy=0, size=20).encode(
    text='fecha:T'
).transform_filter(
    pts
)

#Regression
reg= alt.Chart(df_reg).mark_line(opacity=0.2,point=True).encode(
    alt.X('fecha:T',title='Fecha'),
    alt.Y('fitted:Q',scale=alt.Scale(type='log'),title='#Confirmados Totales en Argentina (log scale)'),
    color=alt.Color('fitted_flag',
                   scale=alt.Scale(
            domain=['extrapolado', 'ajustado'],
            range=['red', 'green']),title='#Confirmados(t)=a*b^t'),
    tooltip=['daily_expected'], 
).properties(        width=550,
        height=400).interactive()


In [ ]:
full_chart=alt.vconcat(points+reg, bars+texts)

In [ ]:
full_chart.configure_axis(
    labelFontSize=15,
    titleFontSize=16
)